# Comparing simulation results

In [1]:
%matplotlib widget

import __init__
import scripts.config as config
import numpy as np
import pandas as pd
import tempfile
import datetime
import ipywidgets as widgets
from ipywidgets import interact
from natsort import natsorted
import os
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib.font_manager import FontProperties
import seaborn as sns
# import matplotlib as mpl
import matplotlib.pyplot as plt
import importlib
from functools import reduce

In [2]:
results_dir = config.velma_data.parents[1] / 'results'
sims = ['ellsworth_baseline_03_19_2', 'ellsworth_ind40_03_19_2']
sim_start = pd.to_datetime('01-01-2003')
sim_end = pd.to_datetime('12-31-2019')
sim_range = pd.date_range(sim_start, sim_end)

dailies = []
annuals = []
for sim in sims:
    sim_dir = results_dir / sim
    files = os.listdir(sim_dir)
    
    daily_results = pd.read_csv(sim_dir / 'DailyResults.csv')
    
    # Format datetime
    jday_pad = daily_results['Day'].apply(lambda x: str(x).zfill(3))
    str_year = daily_results['Year'].apply(lambda x: str(x))
    rng = pd.to_datetime((str_year + jday_pad), format='%Y%j')
    daily_results.index = rng
    dailies.append(daily_results)
    
    annual_results = pd.read_csv(sim_dir / 'AnnualResults.csv')
    annuals.append(annual_results)



### Runoff

In [9]:
# Daily runoff time series (all years together)

# Check that all dfs have same number of columns
all([len(dailies[0].columns.intersection(df.columns)) 
      == dailies[0].shape[1] for df in dailies])

# colors = sns.color_palette('tab10', len(sims))

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(9, 5))

for i, sim in enumerate(sims):
    ax.set_ylim([0, 70])
    dailies[i].loc[:, 'Runoff_All(mm/day)_Delineated_Average'].plot(ax=ax, label=sim, linewidth=1)
leg = ax.legend(loc='upper left', fancybox=True, ncol=3)
for line in leg.get_lines():
    line.set_linewidth(4.0)
ax.set_ylabel('Runoff (mm/day)')
plt.suptitle('Daily runoff')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# Daily runoff by year

runoff_sims = pd.DataFrame()
for i, df in enumerate(dailies):
    runoff_sims[i] = df.loc[:, 'Runoff_All(mm/day)_Delineated_Average']

runoff_sims.columns = sims
runoff_sims['doy'], runoff_sims['year'] = runoff_sims.index.dayofyear, runoff_sims.index.year
runoff_sim_yearly = pd.pivot_table(runoff_sims, index=['doy'], columns=['year'], values=sims, dropna=False)
# runoff_sims_df.columns = sims
# runoff_sims_df = pd.DataFrame(data=runoff_sims_stack, columns=sims, index=rng)
# runoff_sims_df['doy'], runoff_sims_df['year'] = runoff_sims_df.index.dayofyear, runoff_sims_df.index.year


In [11]:
plt.close('all')
years = runoff_sim_yearly.columns.get_level_values(1).unique()
fig, axes = plt.subplots(ncols=1, nrows=len(years), figsize=(8, 22))
for i, year in enumerate(years):
    axes[i].set_title(year)
    axes[i].set_ylim([0, 80])
    for j, sim in enumerate(sims):
        tuples = pd.MultiIndex.from_product([[sim], [year]])
        runoff_sim_yearly[tuples].plot(ax=axes[i], label=sim, linewidth=1)
    axes[i].get_legend().remove()
    leg = axes[0].legend(loc='upper left', bbox_to_anchor=(0, 3), fancybox=True, ncol=3)
    for line in leg.get_lines():
        line.set_linewidth(4.0)
axes[0].set_ylabel('Runoff (mm/day)')
# plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Runoff (mm/day)')

In [6]:
# Plot specific years interactively

plt.close('all')
years = runoff_sim_yearly.columns.get_level_values(1).unique()
def plot_selected_sims(selected):    
    fig, axes = plt.subplots(ncols=1, nrows=len(selected), figsize=(8, len(selected)*3), squeeze=False)
    for i, year in enumerate(selected):
        axes[i, 0].set_title(year)
#         axes[i, 0].set_ylim([0, 80])
        for j, sim in enumerate(sims):
            tuples = pd.MultiIndex.from_product([[sim], [year]])
            runoff_sim_yearly[tuples].plot(ax=axes[i, 0], label=sim, linewidth=1)
        axes[i, 0].get_legend().remove()
    leg = axes[0, 0].legend(loc='upper left', bbox_to_anchor=(0, 1.5), fancybox=True, ncol=3)
    for line in leg.get_lines():
        line.set_linewidth(4.0)
    axes[0, 0].set_ylabel('Runoff (mm/day)')
    plt.suptitle('Daily runoff')
    plt.tight_layout()

sel_mul = widgets.SelectMultiple(description="Simulations", options=years, value=[years[0], years[1]], disabled=False)
@interact(selected=sel_mul)
def interactive_plot(selected):
    tuples = pd.MultiIndex.from_product([selected, years])
#     runoff_sim_yearly_sub = runoff_sim_yearly[tuples]
    plot_selected_sims(selected)

interactive(children=(SelectMultiple(description='Simulations', index=(0, 1), options=(2003, 2004, 2005, 2006,…

In [7]:
# Sum of daily runoff

sums = pd.DataFrame(runoff_sims.drop(['doy', 'year'], axis=1).sum(axis=0))
sums.plot.bar(rot=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [ ]:
# Annual aveage, min runoff
plt.close('all')


runoff_min_annual = pd.DataFrame(columns=sims, index=years)

runoff_min_jday_annual = runoff_min_annual.copy()
runoff_max_annual = runoff_min_annual.copy()
runoff_avg_annual = runoff_min_annual.copy()
for i, df in enumerate(annuals):
    runoff_min_annual[i] = df[df['Annual_Result'] == 'MIN_VALUE'].loc[:, 'Runoff_All(mm/day)_Delineated_Average']
    runoff_min_jday_annual[i] = df[df['Annual_Result'] == 'MIN_VALUE_JDAY'].loc[:, 'Runoff_All(mm/day)_Delineated_Average']
    runoff_max_annual[i] = df[df['Annual_Result'] == 'MAX_VALUE'].loc[:, 'Runoff_All(mm/day)_Delineated_Average']
    runoff_avg_annual[i] = df[df['Annual_Result'] == 'ANNUAL_AVERAGE'].loc[:, 'Runoff_All(mm/day)_Delineated_Average']


# fig, axes = plt.subplots(ncols=1, nrows=4, figsize=(8, 9))
# runoff_avg_annual.plot(ax=axes[0])
# runoff_max_annual.plot(ax=axes[1])
# runoff_min_annual.plot(ax=axes[2])
# runoff_min_jday_annual.plot(ax=axes[3])
# leg = ax.legend(loc='upper left', bbox_to_anchor=(0, 1.2), fancybox=True, ncol=3)
# for line in leg.get_lines():
#     line.set_linewidth(4.0)
# ax.set_ylabel('Runoff (mm/day)')
# plt.suptitle('Annual average runoff')
# plt.tight_layout()

In [ ]:
# Annual aveage runoff
plt.close('all')
annual_metric = pd.DataFrame()
for i, df in enumerate(annuals):
    annual_metric[i] = df[df['Annual_Result'] == 'ANNUAL_AVERAGE'].loc[:, 'Runoff_All(mm/day)_Delineated_Average']

annual_metric.columns = sims
annual_metric.index = years

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(8, 4))
annual_metric.plot(ax=ax)
leg = ax.legend(loc='upper left', bbox_to_anchor=(0, 1.2), fancybox=True, ncol=3)
for line in leg.get_lines():
    line.set_linewidth(4.0)
ax.set_ylabel('Runoff (mm/day)')
plt.suptitle('Annual average runoff')
plt.tight_layout()

In [ ]:
# Annual aveage runoff
plt.close('all')
annual_metric = pd.DataFrame()
for i, df in enumerate(annuals):
    annual_metric[i] = df[df['Annual_Result'] == 'MIN_VALUE'].loc[:, 'Runoff_All(mm/day)_Delineated_Average']

annual_metric.columns = sims
annual_metric.index = years

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(8, 4))
annual_metric.plot(ax=ax)
leg = ax.legend(loc='upper left', bbox_to_anchor=(0, 1.2), fancybox=True, ncol=3)
for line in leg.get_lines():
    line.set_linewidth(4.0)
ax.set_ylabel('Runoff (mm/day)')
plt.suptitle('Annual minimum runoff')
plt.tight_layout()

In [ ]:
plt.close('all')
annual_metric = pd.DataFrame()
for i, df in enumerate(annuals):
    annual_metric[i] = df[df['Annual_Result'] == 'MIN_VALUE_JDAY'].loc[:, 'Runoff_All(mm/day)_Delineated_Average']

annual_metric.columns = sims
annual_metric.index = years

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(8, 4))
annual_metric.plot(ax=ax)
leg = ax.legend(loc='upper left', bbox_to_anchor=(0, 1.2), fancybox=True, ncol=3)
for line in leg.get_lines():
    line.set_linewidth(4.0)
ax.set_ylabel('Runoff (mm/day)')
plt.suptitle('Annual minimum runoff Jday')
plt.tight_layout()

In [ ]:
plt.close('all')
annual_metric = pd.DataFrame()
for i, df in enumerate(annuals):
    annual_metric[i] = df[df['Annual_Result'] == 'MAX_VALUE'].loc[:, 'Runoff_All(mm/day)_Delineated_Average']

annual_metric.columns = sims
annual_metric.index = years

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(8, 4))
annual_metric.plot(ax=ax)
leg = ax.legend(loc='upper left', bbox_to_anchor=(0, 1.2), fancybox=True, ncol=3)
for line in leg.get_lines():
    line.set_linewidth(4.0)
ax.set_ylabel('Runoff (mm/day)')
plt.suptitle('Annual max runoff')
plt.tight_layout()